In [ ]:
import numpy
import glob
import pyfits
import rawpy
import scipy.stats
import os.path
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

bias_folder = "Calibration_Files/D7000Bias/"
flat_folder = "9_23/Flat/"
dark_folder = "9_23/Sadr_Crescent/Dark/"
light_folder = "9_23/Sadr_Crescent/"
Camera = "D7000"
Subtract_dark = 1
Use_flat_field = 1

##  DO NOT MODIFY ANYTHING BELOW THIS LINE  ##

In [ ]:
#Region Definition

if Camera in ["D7000","D5100"]:
    bias1_region_x1 = 4960
    bias1_region_x2 = 4964
    bias1_region_y1 = 70
    bias1_region_y2 = 3358
    bias2_region_x1 = 4964
    bias2_region_x2 = 4968
    bias2_region_y1 = 70
    bias2_region_y2 = 3358
    dark1_region_x1 = 0
    dark1_region_x2 = 2
    dark1_region_y1 = 70
    dark1_region_y2 = 3358
    dark2_region_x1 = 4958
    dark2_region_x2 = 4960
    dark2_region_y1 = 70
    dark2_region_y2 = 3358
    active_region_x1 = 4
    active_region_x2 = 4956
    active_region_y1 = 70
    active_region_y2 = 3358
elif Camera in ["D7000_Per","D5100_Per"]:
    bias1_region_x1 = 4948
    bias1_region_x2 = 4949
    bias1_region_y1 = 0
    bias1_region_y2 = 3280
    bias2_region_x1 = 4949
    bias2_region_x2 = 4950
    bias2_region_y1 = 0
    bias2_region_y2 = 3280
    dark1_region_x1 = 4950
    dark1_region_x2 = 4951
    dark1_region_y1 = 0
    dark1_region_y2 = 3280
    dark2_region_x1 = 4951
    dark2_region_x2 = 4952
    dark2_region_y1 = 0
    dark2_region_y2 = 3280
    active_region_x1 = 0
    active_region_x2 = 4948
    active_region_y1 = 0
    active_region_y2 = 3280
elif Camera in ["D600","D610"]:
    bias1_region_x1 = 2
    bias1_region_x2 = 4
    bias1_region_y1 = 84
    bias1_region_y2 = 4030
    bias2_region_x1 = 6040
    bias2_region_x2 = 6042
    bias2_region_y1 = 84
    bias2_region_y2 = 4030
    dark1_region_x1 = 0
    dark1_region_x2 = 2
    dark1_region_y1 = 84
    dark1_region_y2 = 4030
    dark2_region_x1 = 6042
    dark2_region_x2 = 6046
    dark2_region_y1 = 84
    dark2_region_y2 = 4030
    active_region_x1 = 24
    active_region_x2 = 6040
    active_region_y1 = 88
    active_region_y2 = 4028
elif Camera in ["D800","D800E"]:
    bias1_region_x1 = 2
    bias1_region_x2 = 4
    bias1_region_y1 = 46
    bias1_region_y2 = 4978
    bias2_region_x1 = 7384
    bias2_region_x2 = 7386
    bias2_region_y1 = 46
    bias2_region_y2 = 4978
    dark1_region_x1 = 0
    dark1_region_x2 = 2
    dark1_region_y1 = 46
    dark1_region_y2 = 4978
    dark2_region_x1 = 7386
    dark2_region_x2 = 7398
    dark2_region_y1 = 46
    dark2_region_y2 = 4978
    active_region_x1 = 24
    active_region_x2 = 7384
    active_region_y1 = 46
    active_region_y2 = 4978

In [ ]:
#read RAW bayer
def Get_RAW_np_arr(filename):
    return rawpy.imread(filename).raw_image

#Write to FITS
def Write_np_fits(filename, array):
    hdu = pyfits.PrimaryHDU()
    hdu.data = array
    hdu.writeto(filename, clobber=True)

In [ ]:
#process bias images

if os.path.exists(bias_folder+"Master_Bias.FITS"):
    hdu = pyfits.open(bias_folder+"Master_Bias.FITS")
    master_bias = hdu[0].data
else:
    bias_stack = []
    for item in glob.glob(bias_folder+"*.NEF"):
        bias_stack.append(Get_RAW_np_arr(item))
    bias_stack = numpy.array(bias_stack)
    master_bias = numpy.mean(bias_stack, axis=0, dtype='f4')
    bias_stack = None
    Write_np_fits(bias_folder+"Master_Bias.FITS",master_bias)

bias_region1 = master_bias[bias1_region_y1:bias1_region_y2,bias1_region_x1:bias1_region_x2].flatten()
bias_region2 = master_bias[bias2_region_y1:bias2_region_y2,bias2_region_x1:bias2_region_x2].flatten()
master_bias_offset = numpy.mean(numpy.append(bias_region1, bias_region2).flatten())

print(master_bias_offset)

In [ ]:
#Flat field processing
if Use_flat_field:
    if os.path.exists(flat_folder+"Master_Flat_mask.FITS"):
        hdu = pyfits.open(flat_folder+"Master_Flat_mask.FITS")
        flat_mask = hdu[0].data
    else:
        flat_stack = []
        intensity = []
        for item in glob.glob(flat_folder+"*.NEF"):
            corrected_flat = Get_RAW_np_arr(item).astype('f8') - master_bias
            flat_stack.append(corrected_flat)
            #Get center brightness for normalization...
            h,w = corrected_flat.shape
            yc = h >> 1
            xc = w >> 1
            center = corrected_flat[yc-64:yc+64,xc-64:xc+64]
            center_green = center[::2,::2]
            intensity.append(numpy.mean(center_green))
        print(intensity)
        #normalize and stack...
        ave_signal = numpy.mean(intensity)
        for i,flat_intensity in enumerate(intensity):
            cor_factor = ave_signal/flat_intensity
            flat_stack[i] *= cor_factor
        #Merge flats
        flat_stack = numpy.array(flat_stack)
        master_flat = numpy.mean(flat_stack, axis = 0)
        flat_stack=None
        #Regression Analysis of red/blue channel. We need to keep white balance
        flat_area_effective = master_flat[active_region_y1:active_region_y2,active_region_x1:active_region_x2]
        master_flat = None
        red_f = flat_area_effective[0::2,0::2].flatten()
        green_channel = flat_area_effective[0::2,1::2]
        green_f = green_channel.flatten()
        blue_f = flat_area_effective[1::2,1::2].flatten()
        k_red = numpy.sum(red_f * green_f) / numpy.sum(red_f * red_f)
        k_blue = numpy.sum(blue_f * green_f) / numpy.sum(blue_f * blue_f)

        plt.plot(blue_f, green_f,'.')
        plt.plot([0, numpy.max(green_f)], [0, numpy.max(green_f)*k_blue], 'k-')
        plt.show()
        red_f = None
        green_f = None
        blue_f = None

        h_g = (active_region_y2 - active_region_y1) >> 1
        w_g = (active_region_x2 - active_region_x1) >> 1
        flat_target = numpy.median(green_channel[h_g-64:h_g+64,w_g-64:w_g+64])
        green_channel = None
        #make flat correction mask
        flat_mask = flat_target/flat_area_effective
        flat_area_effective = None
        flat_mask[0::2,0::2] *= 1/k_red
        flat_mask[1::2,1::2] *= 1/k_blue
        flat_mask = flat_mask.astype('f4')
        Write_np_fits(flat_folder+"Master_Flat_mask.FITS",flat_mask)

In [ ]:
#process dark images
dark_stack = []
dark_count = []
#dark_count_effective = []
dark_stack_new = []
for item in glob.glob(dark_folder+"*.NEF"):
    darkframe = Get_RAW_np_arr(item)
    bias_region1 = darkframe[bias1_region_y1:bias1_region_y2,bias1_region_x1:bias1_region_x2].flatten()
    bias_region2 = darkframe[bias2_region_y1:bias2_region_y2,bias2_region_x1:bias2_region_x2].flatten()
    dark_frame_bias_offset = numpy.mean(numpy.append(bias_region1, bias_region2))
    #dark_region1 = darkframe[dark1_region_y1:dark1_region_y2,dark1_region_x1:dark1_region_x2].flatten()
    #dark_region2 = darkframe[dark2_region_y1:dark2_region_y2,dark2_region_x1:dark2_region_x2].flatten()
    #dark_regions = numpy.append(dark_region1, dark_region2)
    #mode = scipy.stats.mode(dark_regions)[0][0]
    
    #Use all effective pixels to normalize dark frames
    dark_effective = darkframe[active_region_y1:active_region_y2, active_region_x1:active_region_x2].flatten()
    counts = numpy.bincount(dark_effective.astype('u2'))
    mode = numpy.argmax(counts)
    
    # Make the y-axis label, ticks and tick labels match the line color.
    
    #dark_count_effective.append(main_mode)
    #dark_effective = darkframe[36:70,0:4948].flatten()
    
    #ax1.set_ylabel('exp', color='b')
    #ax1.tick_params('y', colors='b')
    #fig, ax1 = plt.subplots()
    #ax1.hist(dark_effective,bins=range(128,256,1), histtype="step")
    #ax1.set_xlabel('time (s)')
    #ax2 = ax1.twinx()
    #ax2.hist(dark_regions,bins=range(128,256,1), histtype="step",color='r')
    #ax2.set_ylabel('sin', color='r')
    #ax2.tick_params('y', colors='r')
    #fig.tight_layout()
    #plt.show()
    
    
    dark_frame_dark_count = mode - dark_frame_bias_offset
    print(mode, dark_frame_bias_offset)
    darkframe = darkframe.astype('f8') - master_bias - dark_frame_bias_offset + master_bias_offset
    #print(numpy.mean(darkframe[bias1_region_y1:bias1_region_y2,bias1_region_x1:bias2_region_x2].flatten()))
    dark_stack.append(darkframe)
    dark_count.append(dark_frame_dark_count)
    dark_stack_new.append(darkframe)
dark_stack = numpy.array(dark_stack)

In [ ]:
ave_dark_count = numpy.mean(dark_count)
print("Dark count",dark_count)
print("Mean dark count",ave_dark_count)
dark_normalize_factor = ave_dark_count/numpy.array(dark_count)
for i in range(dark_stack.shape[0]):
    dark_stack[i] = dark_stack[i] * dark_normalize_factor[i]
master_dark = numpy.median(dark_stack, axis=0)
dark_stack = None

#print(dark_count_effective)

#Write_np_fits(Camera+"_Master_Dark.FITS",master_dark.astype("f4"))

In [ ]:
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10,10)

plt.plot(dark_stack_new[0][active_region_y1:active_region_y2, active_region_x1:active_region_x2].flatten(),
         dark_stack_new[3][active_region_y1:active_region_y2, active_region_x1:active_region_x2].flatten(), ',')
plt.plot([0, 10000],[0,10000* dark_normalize_factor[0]/dark_normalize_factor[3]],'k-')
plt.show()
dark_stack_new = None

In [ ]:
#hot pixel rejection
hotpixel_dark = master_dark[active_region_y1:active_region_y2,active_region_x1:active_region_x2]
counts = numpy.bincount(hotpixel_dark.flatten().astype('u2'))
mode = numpy.argmax(counts)
print("Effective pixel dark count mode",mode)
max_count = numpy.max(counts)
for i in range(mode, len(counts)):
    if counts[i] <= (max_count >> 1):
        break
std = (i - mode)*2 /2.3548
hotpixel_thres = mode+std*32
hotpix_coord = numpy.where(hotpixel_dark > hotpixel_thres)
print("Hot pixel threshold at",hotpixel_thres,"count",len(hotpix_coord[0]))

master_dark_region1 = master_dark[dark1_region_y1:dark1_region_y2,dark1_region_x1:dark1_region_x2].flatten()
master_dark_region2 = master_dark[dark2_region_y1:dark2_region_y2,dark2_region_x1:dark2_region_x2].flatten()
master_dark_reference = numpy.append(master_dark_region1, master_dark_region2)

ker = scipy.stats.gaussian_kde(master_dark_reference.flatten())
ker.set_bandwidth(0.02)
x_grid = numpy.arange(20,60,0.1)
density_fit = ker(x_grid)

master_dark_ob_ave = x_grid[numpy.argmax(density_fit)]

#counts = numpy.bincount(master_dark_reference.astype('u2'))
#master_dark_ob_ave = numpy.argmax(counts)

print("Dark count in optical black",master_dark_ob_ave)
fig, ax1 = plt.subplots()
ax1.set_ylabel('Effective', color='b')
ax1.tick_params('', colors='b')

ax1.hist(hotpixel_dark.flatten(),bins=range(20,60,1), histtype="step")
ax1.set_xlabel('Intensity')
ax2 = ax1.twinx()
ax2.plot(x_grid,density_fit*numpy.max(numpy.bincount(master_dark_reference.astype('u2')))/numpy.max(density_fit),color='r')
ax2.hist(master_dark_reference,bins=range(20,60,1),  histtype="step",color='r')
ax2.set_ylabel('OB', color='r')
ax2.tick_params('y', colors='r')
fig.tight_layout()
plt.show()

In [ ]:
#process light frames
for item in sorted(glob.glob(light_folder+"*.NEF")):
    lightframe = Get_RAW_np_arr(item)
    bias_region1 = lightframe[bias1_region_y1:bias1_region_y2,bias1_region_x1:bias1_region_x2].flatten()
    bias_region2 = lightframe[bias2_region_y1:bias2_region_y2,bias2_region_x1:bias2_region_x2].flatten()
    bias_region = numpy.append(bias_region1, bias_region2)
    light_bias_offset = numpy.mean(bias_region[bias_region < 1000])
    cal_light = lightframe.astype('f8') - master_bias - light_bias_offset + master_bias_offset
    dark_region1 = cal_light[dark1_region_y1:dark1_region_y2,dark1_region_x1:dark1_region_x2].flatten()
    dark_region2 = cal_light[dark2_region_y1:dark2_region_y2,dark2_region_x1:dark2_region_x2].flatten()
    dark_regions = numpy.append(dark_region1, dark_region2)
    
    mode = numpy.argmax(numpy.bincount(dark_regions.astype('u2')))
    
    #fitting using kde
    ker = scipy.stats.gaussian_kde(dark_regions)
    ker.set_bandwidth(0.03)
    ker_min = min(dark_regions)-10
    ker_max = numpy.median(dark_regions)*2 - ker_min
    x_grid = numpy.arange(ker_min, ker_max,0.1)
    ker_fit = ker(x_grid)
    mode_with_fit = x_grid[numpy.argmax(ker_fit)]
    
    #plotting
    """fig, ax1 = plt.subplots()
    ax1.set_ylabel('Freq', color='b')
    ax1.hist(dark_regions, bins=range(int(ker_min),int(ker_max)), histtype="step")
    ax1.set_xlabel('Intensity')
    ax2 = ax1.twinx()
    ax2.plot(x_grid, ker_fit)
    ax2.set_ylabel('Gaussian Fitting', color='r')
    fig.tight_layout()
    plt.show()"""
    
    light_dark_count = mode_with_fit
    if Subtract_dark:
        cal_light = cal_light - master_dark * (light_dark_count / master_dark_ob_ave)
    else:
        cal_light -= light_dark_count
    #Crop active area
    """dark_region1 = cal_light[dark1_region_y1:dark1_region_y2,dark1_region_x1:dark1_region_x2].flatten()
    dark_region2 = cal_light[dark2_region_y1:dark2_region_y2,dark2_region_x1:dark2_region_x2].flatten()
    ob_calibrated = numpy.append(dark_region1, dark_region2)"""
    cal_light = cal_light[active_region_y1:active_region_y2,active_region_x1:active_region_x2]
    if Use_flat_field:
        cal_light *= flat_mask
    print(item,"OB dark", mode, mode_with_fit, "Bias", light_bias_offset)
    #Hot pixel interpolation
    for i in range(len(hotpix_coord[0])):
        y = hotpix_coord[0][i]
        x = hotpix_coord[1][i]
        #ignore boundary
        #Naive method, assume no adjacent hot pixel in the same color plane
        if x > 1 and y > 1 and x < active_region_x2-active_region_x1-2 and y < active_region_y2-active_region_y1-2:
            if (y % 2) ^ (x % 2):
                #Green Channel
                cal_light[y,x] = (cal_light[y-1,x-1]+cal_light[y+1,x-1]+cal_light[y+1,x+1]+cal_light[y-1,x+1])/4
            else: #red and blue
                cal_light[y,x] = (cal_light[y-2,x]+cal_light[y,x-2]+cal_light[y+2,x]+cal_light[y,x+2])/4
    cal_light = cal_light*4 + 200
    Write_np_fits(item.replace(".NEF","_cal.FITS"), cal_light.astype('u2'))